A Hamiltonian cycle in a graph is a cycle which contains every vertex. Let $\mathcal{G}(n, p)$ be the space of graphs with $n$ labelled vertices and edges appearing independently at random with probability $p$.

To determine if a graph has a Hamiltonian cycle, we can use a backtracking algorithm. This method builds a path, one vertex at a time, and if the path cannot be extended to a full cycle, it "backtracks" to try a different choice.

1.  Begin at an arbitrary vertex and add it to the current path.
2.  From the last vertex in the path, try to add one of its unvisited neighbors to the path.
3.  If a valid neighbor is found, add it to the path and repeat step 2 recursively for the new, longer path.
4.  If the path contains all $n$ vertices, check if the last vertex is connected to the starting vertex. If it is, a Hamiltonian cycle has been found.
5.  If the path cannot be extended (i.e., the current vertex has no unvisited neighbors) or if a recursive call fails, remove the current vertex from the path and return to the previous vertex to try another one of its neighbors.
6.  If all possibilities have been explored without finding a cycle, then the graph has no Hamiltonian cycle.

This algorithm explores all potential paths in a depth-first search (DFS). It is guaranteed to find a cycle if one exists but its worst-case time complexity is factorial $O(n!)$, making it computationally expensive for large graphs.

In [17]:
import random
import math

def generate_graph(n, p):
    '''
    Generates a random graph from G(n, p).
    '''
    if n <= 0:
        return []
    # Initialize an n x n adjacency matrix with zeros
    graph = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            # For each pair of vertices, add an edge with probability p
            if random.random() < p:
                graph[i][j] = 1
                graph[j][i] = 1
    return graph

def has_hamiltonian_cycle(graph):
    '''
    Checks if a graph has a Hamiltonian cycle using backtracking.
    '''
    n = len(graph)
    if n < 3:
        return False
    path = [-1] * n
    path[0] = 0  # Start the path from the first vertex
    # Call the recursive function to find the cycle
    if not _hamiltonian_util(graph, path, 1):
        return False
    return True

def _hamiltonian_util(graph, path, pos):
    '''
    A recursive function to solve the Hamiltonian cycle problem.
    '''
    n = len(graph)
    # Base case: If all vertices are included in the path
    if pos == n:
        # Check if there is an edge from the last vertex to the first vertex
        if graph[path[pos - 1]][path[0]] == 1:
            return True
        else:
            return False
    # Try different vertices as the next candidate in the path
    for v in range(1, n):
        if _is_safe(v, graph, path, pos):
            path[pos] = v
            # Recur to construct the rest of the path
            if _hamiltonian_util(graph, path, pos + 1):
                return True
            # If adding vertex v doesn't lead to a solution, backtrack
            path[pos] = -1
    return False

def _is_safe(v, graph, path, pos):
    '''
    A function to check if a vertex can be added at the given position.
    '''
    # Check if this vertex is an adjacent vertex of the previously added vertex.
    if graph[path[pos - 1]][v] == 0:
        return False
    # Check if the vertex has already been included in the path.
    if v in path:
        return False
    return True

def run_experiment(n_values, p_generator, num_samples):
    '''
    Runs an experiment by generating graphs and checking for Hamiltonian cycles.
    Args:
        n_values: A list of n values to test.
        p_generator: A function that takes n and returns a list of p values.
        num_samples: The number of random graphs to generate for each (n, p) pair.

    Returns:
        A dictionary with the results and a list of the p_labels used.
    '''
    results = {}
    all_p_labels = []
    for n in n_values:
        p_values, p_labels = p_generator(n)
        if not all_p_labels:
             all_p_labels = p_labels
        results[n] = {}
        for p, p_label in zip(p_values, p_labels):
            hamiltonian_count = 0
            for _ in range(num_samples):
                g = generate_graph(n, p)
                if has_hamiltonian_cycle(g):
                    hamiltonian_count += 1
            results[n][p_label] = hamiltonian_count
    return results, all_p_labels

def print_table(title, results, p_labels):
    '''
    Prints a formatted table of results.
    '''
    print(f'{title}')
    header = f'{"n\\p":<12}' + ''.join([f'{label:<12}' for label in p_labels])
    print('-' * len(header))
    print(header)
    print('-' * len(header))

    for n, p_results in sorted(results.items()):
        row_str = f'{n:<12}'
        for p_label in p_labels:
            count = p_results.get(p_label, 'N/A')
            row_str += f'{str(count):<12}'
        print(row_str)
    print('\n')

The worst-case scenario for this algorithm occurs when it must explore the largest possible number of paths before finding a cycle or determining that none exists. This typically happens with graphs that have many paths that are "almost" Hamiltonian cycles, forcing the algorithm to go deep into its search before backtracking. An example of a worst-case graph is a complete graph $K_n$, where every vertex is connected to every other vertex.

We fix the starting vertex $0$. From the starting vertex, there are $n-1$ possible choices for the next vertex in the path. From the second vertex, there are $n-2$ remaining choices. This continues until the last vertex, where there is only 1 choice left. The total number of permutations of vertices to check is $(n-1)!$.

For each permutation, the algorithm performs a constant number of operations of order $O(n)$ (checking for an edge, checking if a vertex has been visited).
Therefore, the total complexity of a worst-case running time of is factorial $O(n!)$.

The average-case running time depends heavily on the structure of the graph, particularly its density.

*   In a sparse graph, most vertices have a low degree. The backtracking algorithm will quickly discover that it cannot extend the current path because there are no available unvisited neighbors. This leads to frequent and early backtracking. As a result, the search tree is pruned very effectively, and the running time is much better than the worst case. The algorithm will likely not explore many full-length paths.

*   In a dense graph, the high number of edges means that a Hamiltonian cycle is very likely to exist. The algorithm will still explore many paths, but it is likely to find a valid cycle relatively quickly without needing to exhaust the entire search space. The expected running time is therefore better than the worst case because the search is often successful long before all possibilities are checked.

*   The most challenging cases lie in a transition region, where the probability $p$ is just around the threshold for a Hamiltonian cycle to appear (which for large $n$ is around $p = (\log(n) + \log\log(n)) / n$. The graphs might have many long paths but no complete cycle, forcing the algorithm to do a significant amount of work. This should still be empirically much faster than $O(n!)$ for most random graphs as the number of paths that need to be explored is a tiny fraction of the $(n-1)!$ total possibilities.

In [18]:
# Parameters
MAX_N = 12
N_VALUES = range(5, MAX_N + 1)
NUM_SAMPLES = 20  # Number of graphs to test for each (n, p) pair

# 1. p varies from 0.1 to 0.9
def p_gen_exp1(n):
    p_values = [i / 10.0 for i in range(1, 10)]
    p_labels = [f'{p:.1f}' for p in p_values]
    return p_values, p_labels

results1, labels1 = run_experiment(
    n_values = N_VALUES,
    p_generator = p_gen_exp1,
    num_samples = NUM_SAMPLES
)
print_table(
    f'Number of graphs with a Hamiltonian cycle (out of {NUM_SAMPLES})',
    results1,
    labels1
)

# 2. p varies from 0.1*ln(n)/n to 1.9*ln(n)/n
def p_gen_exp2(n):
    p_values = [c * 0.1 * math.log(n) / n for c in range(1, 20, 2)]
    p_labels = [f'{c * 0.1:.1f}ln(n)/n' for c in range(1, 20, 2)]
    return p_values, p_labels

results2, labels2 = run_experiment(
    n_values = N_VALUES,
    p_generator = p_gen_exp2,
    num_samples = NUM_SAMPLES
)
print_table(
    f'Number of graphs with a Hamiltonian cycle (out of {NUM_SAMPLES})',
    results2,
    labels2
)

Number of graphs with a Hamiltonian cycle (out of 20)
------------------------------------------------------------------------------------------------------------------------
n\p         0.1         0.2         0.3         0.4         0.5         0.6         0.7         0.8         0.9         
------------------------------------------------------------------------------------------------------------------------
5           0           0           0           2           7           7           15          16          19          
6           0           0           0           2           8           12          18          20          20          
7           0           0           0           2           9           12          17          20          20          
8           0           0           0           6           10          18          19          20          20          
9           0           0           2           3           14          18          20          20 

A sufficient but not necessary property possessed by many non-Hamiltonian graphs is that the graph has a vertex with a degree of less than $2$.

For a Hamiltonian cycle to exist, every single vertex must be part of the cycle where each vertex has exactly two edges that belong to the cycle, one edge to enter and one edge to leave. Therefore, every vertex in a Hamiltonian graph must have a degree of at least $2$ or isolated vertices and leaves are forbidden.

In the first experiment, with a low probability $p$, the resulting graphs are relatively sparse. It is highly probable that some vertices will have very few connections, making vertices of degree $0$ or $1$ quite common implying the graph is non-Hamiltonian.

A graph can have a minimum degree of 2 for all its vertices and still be non-Hamiltonian, for instance two disconnected cycles. A non-trivial example is an hourglass graph with a "cut vertex", i.e., a single vertex whose removal would split the graph into two disconnected components.

The second range of values for $p$ was chosen specifically to observe the phenomenon of a phase transition. There is a critical probability around which the likelihood of a graph having a Hamiltonian cycle shifts from almost $0$ to almost $1$. For a $\mathcal{G}(n, p)$ random graph, this threshold is at $p \approx \log(n) / n$.

The probability $p = \log(n)/n$ is also the threshold at which a random graph is likely to eliminate all vertices of degree less than $2$. If $p$ is significantly less than $\log(n)/n$, then the graph will almost certainly have leaves or isolated vertices. Once $p$ is significantly greater than $\log(n)/n$, the graph's minimum degree will almost certainly be at least $2$.